# Проект по SQL

**Задача** 

Проанализировать базу данных сервиса для чтения книг по подписке.


*Данные*:

    - Таблица books. Содержит данные о книгах.
    - Таблица authors. Содержит данные об авторах.
    - Таблица publishers. Содержит данные об издательствах.
    - Таблица ratings. Содержит данные о пользовательских оценках книг
    - Таблица reviews. Содержит данные о пользовательских обзорах
    

*Схема данных*

![Схема данных](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

*Ход исследования*

- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определим автора с самой высокой средней оценкой книг — учьтем только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Информация о данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'
    } # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# выведем первые строки таблиц

for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = ''' SELECT * FROM {}; '''.format(table)    

    request = pd.io.sql.read_sql(query, con =  con) 

    print(f'Таблица - "{table}"')
    display(request.head())
    print('\n')

Таблица - "books"


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




Таблица - "authors"


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd




Таблица - "publishers"


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company




Таблица - "ratings"


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




Таблица - "reviews"


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитаем, сколько книг вышло после 1 января 2000 года

In [4]:
# создадим sql запрос
query_book_cnt = '''

SELECT COUNT(book_id) 
FROM books 
WHERE CAST(publication_date AS date) >= '2000-01-01';

'''
con=engine.connect()
pd.io.sql.read_sql(query_book_cnt, con =  con) 

,count
0,821


*Комментарий* После 1 января 2000 года вышла 821 книга.

## Для каждой книги посчитаем  количество обзоров и среднюю оценку

In [5]:
# создадим sql запрос
query_rw_rt = '''

SELECT b.title,
       b.book_id,
       COUNT(DISTINCT rw.review_id) AS count_review,
       ROUND(AVG(rt.rating)) AS mean_rating
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
GROUP BY b.title, b.book_id
ORDER BY count_review DESC;            

'''
con=engine.connect()
pd.io.sql.read_sql(query_rw_rt, con =  con) 

,title,book_id,count_review,mean_rating
0,Twilight (Twilight #1),948,7,4.0
1,Water for Elephants,963,6,4.0
2,The Glass Castle,734,6,4.0
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.0
4,The Curious Incident of the Dog in the Night-Time,695,6,4.0
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,4.0
996,The Natural Way to Draw,808,0,3.0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.0
998,Essential Tales and Poems,221,0,4.0


*Комментарий* Посчитали для каждой книги количество обзоров и среднюю оценку. Количество обзоров варьируется от 7 до 0, средний рейтинг от 5 до 2.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [6]:
# создадим sql запрос
query_pb = '''

SELECT p.publisher,
       COUNT(book_id) AS total_books
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher
ORDER BY total_books DESC
LIMIT 1;

'''
con=engine.connect()
pd.io.sql.read_sql(query_pb, con =  con) 

,publisher,total_books
0,Penguin Books,42


*Комментарий* Издательство Penguin Books выпустило больше всего книг - 42 шт.

## Определим автора с самой высокой средней оценкой книг — учьтём только книги с 50 и более оценками

In [7]:
# создадим sql запрос
query_au_rt = '''

SELECT a.author,
       mean_rating
FROM (SELECT b.author_id,
             AVG(rating) AS mean_rating,
             COUNT(rating_id)
      FROM books AS b
      LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
      GROUP BY b.author_id
      HAVING COUNT(rating_id) > 50
      ORDER BY COUNT(rating_id) DESC) AS rt
LEFT JOIN authors AS a ON rt.author_id=a.author_id
GROUP BY a.author, mean_rating
ORDER BY mean_rating DESC
LIMIT 1;


'''
con=engine.connect()
pd.io.sql.read_sql(query_au_rt, con =  con) 

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.288462


*Комментарий* Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку - 4.288

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [8]:
# создадим sql запрос
query_rw_us = '''

SELECT AVG(rw.review_cnt) AS mean_review
FROM (SELECT COUNT(review_id) AS review_cnt,
             username
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(ratings) > 48)
      GROUP BY username) AS rw;
'''
con=engine.connect()
pd.io.sql.read_sql(query_rw_us, con =  con) 

,mean_review
0,24.0


*Комментарий* Среднее количество обзоров пользователей - 24.

## Вывод

Во время исследования данных сервиса для чтения книг по подписке опиралась на 5 таблиц: books, authors, publishers, ratings, reviews. 

Итоги исследования

    После 1 января 2000 года вышла 821 книга.
    Посчитали для каждой книги количество обзоров и среднюю оценку. Количество обзоров варьируется от 7 до 0, средний рейтинг от 5 до 2.
    Издательство Penguin Books выпустило больше всего книг - 42 шт.
    Автор J.K. Rowling/Mary GrandPré имеет самую высокую среднюю оценку - 4.288
    Среднее количество обзоров пользователей - 24.
    
